# Deep Self Ensemble: 8 Way

Train a DNN on the imput text to learn which model performs best on given questions.

### Step 1: Set up environment 

In [ ]:
#Imports 

import datetime
import json
import os
import time
import pprint
import random
import string
import sys
import tensorflow as tf
import re
from collections import Counter
from itertools import groupby
import pandas as pd
import torch.nn as nn

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 6.5MB/s 
     |████████████████████████████████| 890kB 29.7MB/s 
     |████████████████████████████████| 3.0MB 19.4MB/s 
     |████████████████████████████████| 1.1MB 57.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b38bb60ba74046287c85b66265b8fd418342bd7bc8d2f61a23c0be204c36ac79
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import logging
#logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')

In [ ]:
#This will clone the BERT Repo
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 317.20 KiB | 1.81 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [ ]:
#Make sure were in the right place
%ls

bert/  sample_data/


In [ ]:
# Move to BERT folder 
%cd bert

/content/bert


In [ ]:
#Download the SQUAD train and dev dataset
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-07-13 17:07:34--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  94.2MB/s    in 0.4s    

2020-07-13 17:07:34 (94.2 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-07-13 17:07:35--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       

### Step 2: Extract the Text

Need to Extract:
*   Question
*   Context

Then add the [CLS] and [SEP] tokens

In [ ]:
#this will be used to get the full answer list. 
def extract_question_text(data):
    text = []
    for i in range(len(data['data'])):
        paragraphs = len(data['data'][i]['paragraphs'])
        for j in range(paragraphs):
            qas = len(data['data'][i]['paragraphs'][j]['qas'])
            context = data['data'][i]['paragraphs'][j]['context']
            for k in range(qas):
              question = data['data'][i]['paragraphs'][j]['qas'][k]['question']
              text.append([question, context])
    return text

# Need a helper function to open each file
def open_json(path):
    with open(path) as json_file:
        temp_json = json.load(json_file)
        return temp_json

In [ ]:
full_train_data = open_json('train-v2.0.json')
full_text = extract_question_text(full_train_data)
len(full_text)

130319

In [ ]:
full_text[0]

['When did Beyonce start becoming popular?',
 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".']

### Step 3: Tokenize the text

In [ ]:
def add_special(text_list):
  marked_text = []
  for ques, context in text_list:
    temp_text = "[CLS] " + ques + " [SEP]" + context
    marked_text.append(temp_text)
  return marked_text

marked_text = add_special(full_text)
marked_text[0]

['[CLS] When did Beyonce start becoming popular? [SEP]Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".']

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in marked_text:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                        max_length = 200,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',
                        truncation = True     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
#labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', marked_text[0])
print('Token IDs:', input_ids[0])

Original:  [CLS] When did Beyonce start becoming popular? [SEP]Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Token IDs: tensor([  101,  1332,  1225, 24896,  1320,  2093,  1838,  2479,  1927,   136,
          102, 24041,   144, 22080, 25384,   118,  5007,   113,   120,   100,
          120, 17775,   118,   162, 11414,   118,  1474,   114,   113,  125

In [ ]:
#write the answers to CSV so I can pick up where I left off. 

answers_df = pd.read_csv('coded_preds_0.csv')
answers_df = answers_df.drop(['Unnamed: 0','keys'], axis = 1)
answer_array = answers_df.to_numpy()
labels = torch.from_numpy(answer_array)
print(labels.dtype)
print(input_ids.dtype)

torch.int64
torch.int64


### Step 4: Combine with labeled Data to create the dataset

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

117,287 training samples
13,032 validation samples


In [ ]:
x_train = train_dataset[:][0]
y_train = train_dataset[:][1]

In [ ]:
x_train[0]

tensor([  101,  1731,  1242, 16176,  1127,  2536,  1206,  1478,  1105,  1384,
          136,   102,  1109,  1148,  5989,  1104,  1103,  1248,   118,  3964,
         1449,   117,  3291,  8223, 11192,   118, 26528,  1108,  2536,  1107,
         1384,   119,  1135,  1108,  1723,  1118,  1748,  2551, 16176,  1219,
         1371,   118,  1349,   117, 11190,  8458,  2918,  5811,   119,   138,
         1703,  1104,  1479, 16176,  1127,  2536,  1219,  1142,  4065,   119,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

### Step 5: Build DNN

In [ ]:
# If there's a GPU available
if torch.cuda.is_available():        
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size) # Evaluate with this batch size.

In [ ]:

class Model(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.linear = nn.Linear(input_size, 1000)
    self.linear2 = nn.Linear(1000, 1000)
    self.linear3 = nn.Linear(1000, 1000)
    self.linear4 = nn.Linear(1000, output_size)

  def forward(self, x):
    x = torch.relu(self.linear(x))
    x = torch.relu(self.linear2(x))
    x = torch.relu(self.linear3(x))
    x = torch.sigmoid(self.linear4(x))
    return x

  def predict(self, x):
    pred = self.forward(x)
    if pred >= 0.5:
      return 1
    else:
      return 0



In [ ]:
model = Model(200, 8)
model.cuda()
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Step 6: Train and predict

In [ ]:
#training loop

epochs = 100
losses = []
x_train, y_train = x_train.cuda(), y_train.cuda()
for i in range(epochs):
  y_pred = model.forward(x_train.float())
  loss = criterion(y_pred, y_train)
  #print("epoch:", i, "loss", loss.item())
  losses.append(loss.item())
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if i % 10 == 0:
    print("epoch:", i, "loss", loss.item())
    print('Prediction', y_pred[100])
    print(labels[100])

epoch: 0 loss 0.5546018481254578
Prediction tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.6517e-33, 9.9991e-01, 1.0000e+00,
        0.0000e+00, 1.0000e+00], device='cuda:0', grad_fn=<SelectBackward>)
tensor([1, 1, 1, 0, 1, 0, 0, 1])
epoch: 10 loss 0.38011565804481506
Prediction tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([1, 1, 1, 0, 1, 0, 0, 1])
epoch: 20 loss 0.38011565804481506
Prediction tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([1, 1, 1, 0, 1, 0, 0, 1])
epoch: 30 loss 0.38011565804481506
Prediction tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([1, 1, 1, 0, 1, 0, 0, 1])
epoch: 40 loss 0.38011565804481506
Prediction tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([1, 1, 1, 0, 1, 0, 0, 1])
epoch: 50 loss 0.38011565804481506
Prediction tensor([0., 0., 0., 0., 0., 0., 0., 0.], 

### Step 7
Apply the weights to the predictions

In [3]:
"""This was unnecessary since all attempts led to a model
that converged on predicting 0 for all answers at all times.
The results would have been 0% accuracy.  But this inspiried the
BERT for multiple choice situation"""

'This was unnecessary since all attempts led to a model\nthat converged on predicting 0 for all answers at all times.\nThe results would have been 0% accuracy.  But this inspiried the\nBERT for multiple choice situation'